# Azure OpenAI Assistants API Solution

## Load Azure Configuration

In [1]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Functions

In [2]:
tools_function = [
    {   "type": "file_search"},
    {
        "type": "function",
        "function": {
            "name": "get_destinations",
            "description": "Provides a list of hiking destinations or hiking countries."
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_hiking_budget",
            "description": "Provides the typical hiking budget for a country. Use this when money is asked or mentioned in the conversation."
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Provides the weather for a country",
            "parameters": {
                "type": "object",
                "properties": {
                    "country": {
                        "type": "string",
                        "description": "The country (e.g., 'US', 'Australia')."
                    }
                },
                "required": ["country"]
            }
        }
    },

]

# define functions
def get_destinations():
    return """
        United States
        Australia
        France
        """

def get_hiking_budget():
        return """
        If country is not defined:
            Look at all the budgets per country
        United States Budget:
                Budget Hikers: Around $50 per day. This includes camping, cooking your own meals, and using public transportation.
                Mid-Range Hikers: Approximately $150 per day. This covers mid-range accommodations, dining at average restaurants, and some paid attractions.
                Luxury Hikers: About $400 per day. This includes luxury lodges, fine dining, and private transportation.
        Australia Budget:
                Budget Hikers: Around $40 per day. This includes camping, cooking your own meals, and using public transport.
                Mid-Range Hikers: Approximately $120 per day. This covers mid-range accommodations, dining at average restaurants, and some paid attractions.
                Luxury Hikers: About $350 per day. This includes luxury lodges, fine dining, and private transportation.
        France Budget:
                Budget Hikers: Around $45 per day. This includes camping, cooking your own meals, and using public transportation.
                Mid-Range Hikers: Approximately $130 per day. This covers mid-range accommodations, dining at average restaurants, and some paid attractions.
                Luxury Hikers: About $380 per day. This includes luxury lodges, fine dining, and private transportation.            
        """
    
def get_weather(country):
    if country == "United States":
        return """
            The United States has a diverse climate. In general:
            - **Winter**: Cold in the north, mild in the south. Average temperatures range from 26.6°F (-3°C) in Alaska to 70.7°F (21.5°C) in Florida.
            - **Summer**: Hot and humid in the south, dry in the west. Average temperatures range from 52.7°F (11.5°C) to 70.7°F (21.5°C).
            """
    elif country == "Australia":
        return """
                Australia experiences varied climates:
                - **Winter**: Mild in the north, cooler in the south. Average temperatures range from 46°F (8°C) in Canberra to 77°F (25°C) in Darwin.
                - **Summer**: Hot and dry in the interior, humid in the north. Average temperatures range from 68°F (20°C) in Hobart to 91°F (33°C) in Darwin.
            """
    elif country == "France":
        return """
                France has several climate zones:
                - **Winter**: Cold in the north, mild in the south. Average temperatures range from 35.5°F (2°C) in the northeast to 48°F (9°C) in the south.
                - **Summer**: Warm and sunny. Average temperatures range from 63°F (17°C) in the north to 77°F (25°C) along the Mediterranean coast.
            """
    else:
        return "Destination not found"

## Prepare Files

In [3]:
from openai import AzureOpenAI

# Create a client
client = AzureOpenAI(
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key
)

# Create a vector store
vector_store = client.beta.vector_stores.create(name="Hiking Products")

# Specify the folder containing the files
folder_path = "../Data/products/"

# Get all file paths in the folder
file_paths = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path)]

# Open file streams
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

In [ ]:
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)
print(vector_store.id)

## Reformat citations with the proper filenames

In [5]:
def reformat_citations(content_block):
    # Extract the annotations
    annotations = content_block.text.annotations
    
    # Original response
    paragraph = content_block.text.value

    # Dictionary to store key-value pairs of text and filename
    text_filename_pairs = {}

    # Iterate over the annotations and extract the relevant information
    for annotation in annotations:
        file_id = annotation.file_citation.file_id
        text = annotation.text
        cited_file = client.files.retrieve(file_id)
        filename = cited_file.filename

        if text not in text_filename_pairs:
            text_filename_pairs[text] = []
        text_filename_pairs[text].append(filename)

    # Replace the citation texts with their corresponding filenames prefixed with " Source: "
    for text, filenames in text_filename_pairs.items():
        sources = " Source: " + ", ".join(filenames)
        paragraph = paragraph.replace(text, sources)

    return paragraph


## Step 1-2:
1. Create an Assistant
2. Create a Thread

In [ ]:
# Step 1: Create assistant
assistant = client.beta.assistants.create(
  name="Hiking Assistant",
  instructions="""
  You are a assistant that provides information. 
   You will answer questions based on files and tools provided to you about hiking information and products. 
   You will not provide answers outside of those files and tools.
  """,
  model=azure_openai_deployment,
  tools=tools_function,
  tool_resources={"file_search":{"vector_store_ids":[vector_store.id]}},
  temperature=1,
  top_p=1
)

# Step 2: Create thread
thread = client.beta.threads.create()
print(thread)

## Step 3-6: Helper Function
3. Add a message to the thread
4. Run the Assistant
5. Check the Run Status
6. Display the Assistant's Response

In [7]:
import time
import json

def run_assistant(user_question):
  # Step 3: Add a question to the thread
  messages = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_question
  )

  # Step 4: Run the Assistant
  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
  )

  # Step 5: Check the Run Status
  # Looping until the run completes or fails
  while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
      thread_id=thread.id,
      run_id=run.id
    )

    #display run status
    print(run.status)

    if run.status == 'completed':
      messages = client.beta.threads.messages.list(thread_id=thread.id)

      # Step 6: Display the Assistant's Response
      content_block = messages.data[0].content[0]
      annotations = content_block.text.annotations
      if annotations is None:
        value = content_block.text.value
        print(value)
      else:
        print(reformat_citations(content_block))
    
    elif run.status == 'requires_action':
      tool_outputs = []
      for tool_call in run.required_action.submit_tool_outputs.tool_calls:
        function_name = tool_call.function.name
        print("Function Name: ", function_name)
        function_args = json.loads(tool_call.function.arguments)
        print("Function Arguments: ", function_args)    
        
        if function_name == "get_destinations":
          function_response = get_destinations()
        elif function_name == "get_hiking_budget":
          function_response = get_hiking_budget()
        elif function_name == "get_weather":
          function_response = get_weather(country=function_args.get("country"))    
        else:
          function_response = "Unknown function"
        print("Function Response: ", function_response)

        tool_outputs.append(
          { 
            "tool_call_id": tool_call.id,
            "output": str(function_response)
          }
        )
        print("Tool output: ", tool_outputs)
        run = client.beta.threads.runs.submit_tool_outputs(
          thread_id=thread.id,
          run_id=run.id,
          tool_outputs = tool_outputs
        )

    elif run.status == "failed":
        print(run.last_error)
        
    else:
        pass

In [ ]:
# Ask a question
user_question ="""What is the price of the SummitClimber Backpack?"""
run_assistant(user_question)

## Step 7: Append Messages

In [ ]:
user_question ="""What countries can I hike?"""
run_assistant(user_question)

In [ ]:
user_question ="""If I go to the US, how much money do I need for a 3 day hike?"""
run_assistant(user_question)

In [ ]:
user_question ="""What is the weather like there?"""
run_assistant(user_question)

In [ ]:
user_question ="""I have a total budget of $1000. 
If I will hike in the US for 3 days on a budget, what products can you recommend that I buy with the money left?
"""
run_assistant(user_question)

In [ ]:
user_question ="""What country can I go to for 4 days with a budget of $165?"""
run_assistant(user_question)

## Delete Assistant

In [13]:
response = client.beta.assistants.delete(assistant.id)